# ARCOS Data Processing: Parquet to County-Year Aggregation

This notebook processes the filtered ARCOS parquet file and aggregates it to county-year level.

## Prerequisites:
- Run `Preprocessing_Prescription.ipynb` first to create `arcos_filtered_2006_2015.parquet`

## Goals:
1. Load the filtered parquet file
2. Perform comprehensive data quality checks
3. Clean county names
4. Remove invalid/missing values
5. Aggregate to county-year level
6. Validate final dataset
7. Save processed data

## Step 1: Check System Resources

In [1]:
import os

# Check available CPU threads
num_threads = os.cpu_count()
print(f"System Information:")
print(f"  Total CPU threads available: {num_threads}")
print(f"  Recommendation: Use 25-50% of available threads for memory-intensive tasks")
print(f"  Suggested setting: {max(2, num_threads // 4)} threads")

System Information:
  Total CPU threads available: 22
  Recommendation: Use 25-50% of available threads for memory-intensive tasks
  Suggested setting: 5 threads


## Step 2: Load Filtered Parquet File

In [2]:
import polars as pl
import os

# Configure Polars to use fewer threads to avoid crashing
# Adjust this number based on your system (from Step 1 recommendation)
os.environ["POLARS_MAX_THREADS"] = "6"

filtered_file = 'arcos_filtered_2006_2015.parquet'

print(f"Loading data from Parquet: {filtered_file}")
print("=" * 60)
print(f"Polars configured to use 6 threads")

df_arcos = pl.read_parquet(filtered_file)

print(f"\n✓ Data loaded from Parquet!")
print(f"  Rows: {df_arcos.shape[0]:,}")
print(f"  Columns: {df_arcos.shape[1]}")

print("\n" + "=" * 60)
print("Ready to proceed with data quality checks and cleaning...")
print("=" * 60)

Loading data from Parquet: arcos_filtered_2006_2015.parquet
Polars configured to use 6 threads

✓ Data loaded from Parquet!
  Rows: 218,477,461
  Columns: 11

Ready to proceed with data quality checks and cleaning...

✓ Data loaded from Parquet!
  Rows: 218,477,461
  Columns: 11

Ready to proceed with data quality checks and cleaning...


## Step 3: Initial Data Quality Check

In [3]:
print("=" * 60)
print("STEP 3: INITIAL DATA QUALITY CHECK")
print("=" * 60)

# 1. Check data shape
print(f"\n1. Data Shape:")
print(f"   Rows: {df_arcos.shape[0]:,}")
print(f"   Columns: {df_arcos.shape[1]}")

# 2. Check column names and types
print(f"\n2. Column Information:")
print(df_arcos.schema)

# 3. Check for null values
print(f"\n3. Null Values Check:")
null_counts = df_arcos.null_count()
print(null_counts)

# 4. Check year range
print(f"\n4. Year Range:")
year_stats = df_arcos.select([
    pl.col("year").min().alias("min_year"),
    pl.col("year").max().alias("max_year"),
    pl.col("year").n_unique().alias("unique_years")
])
print(year_stats)

# 5. Check states
print(f"\n5. States in Data:")
states = df_arcos.select("BUYER_STATE").unique().sort("BUYER_STATE")
print(f"   Unique states: {states.shape[0]}")
print(f"   States: {states.to_series().to_list()}")

# 6. Check for negative or zero values in key columns
print(f"\n6. Value Ranges Check:")
value_checks = df_arcos.select([
    (pl.col("MME") <= 0).sum().alias("MME_zero_or_neg"),
    (pl.col("DOSAGE_UNIT") <= 0).sum().alias("DOSAGE_UNIT_zero_or_neg"),
    pl.col("MME").min().alias("MME_min"),
    pl.col("MME").max().alias("MME_max"),
    pl.col("DOSAGE_UNIT").min().alias("DOSAGE_min"),
    pl.col("DOSAGE_UNIT").max().alias("DOSAGE_max")
])
print(value_checks)

print(f"\n{'=' * 60}")
print("✓ Initial quality check complete!")

STEP 3: INITIAL DATA QUALITY CHECK

1. Data Shape:
   Rows: 218,477,461
   Columns: 11

2. Column Information:
Schema({'BUYER_BUS_ACT': String, 'BUYER_STATE': String, 'BUYER_COUNTY': String, 'DRUG_NAME': String, 'MME_Conversion_Factor': Float64, 'TRANSACTION_DATE': String, 'Reporter_family': String, 'CALC_BASE_WT_IN_GM': Float64, 'DOSAGE_UNIT': Float64, 'MME': Float64, 'year': Int32})

3. Null Values Check:
shape: (1, 11)
┌─────────────┬─────────────┬─────────────┬───────────┬───┬─────────────┬─────────────┬─────┬──────┐
│ BUYER_BUS_A ┆ BUYER_STATE ┆ BUYER_COUNT ┆ DRUG_NAME ┆ … ┆ CALC_BASE_W ┆ DOSAGE_UNIT ┆ MME ┆ year │
│ CT          ┆ ---         ┆ Y           ┆ ---       ┆   ┆ T_IN_GM     ┆ ---         ┆ --- ┆ ---  │
│ ---         ┆ u32         ┆ ---         ┆ u32       ┆   ┆ ---         ┆ u32         ┆ u32 ┆ u32  │
│ u32         ┆             ┆ u32         ┆           ┆   ┆ u32         ┆             ┆     ┆      │
╞═════════════╪═════════════╪═════════════╪═══════════╪═══╪══════════

## Step 4: Clean County Names with Validation

In [4]:
print("=" * 60)
print("STEP 4: CLEAN COUNTY NAMES")
print("=" * 60)

# Show sample of original county names
print("\n1. Sample of ORIGINAL county names (before cleaning):")
sample_counties_before = df_arcos.select("BUYER_COUNTY").unique().sort("BUYER_COUNTY").head(20)
print(sample_counties_before)

# Check for various issues
print("\n2. County Name Issues:")
county_checks = df_arcos.select([
    (pl.col("BUYER_COUNTY").str.contains("(?i)county")).sum().alias("has_county_suffix"),
    (pl.col("BUYER_COUNTY").str.strip_chars() != pl.col("BUYER_COUNTY")).sum().alias("has_whitespace"),
    pl.col("BUYER_COUNTY").n_unique().alias("unique_counties_before")
])
print(county_checks)

# Clean county names
print("\n3. Cleaning county names...")
df_arcos = df_arcos.with_columns([
    pl.col("BUYER_COUNTY")
    .str.strip_chars()  # Remove leading/trailing whitespace
    .str.to_uppercase()  # Convert to uppercase for consistency
    .str.replace(r"(?i)\s+COUNTY\s*$", "")  # Remove "COUNTY" suffix (case-insensitive)
    .str.strip_chars()  # Remove any trailing whitespace after removal
    .alias("BUYER_COUNTY")
])

print("✓ Cleaning applied!")

# Show sample of cleaned county names
print("\n4. Sample of CLEANED county names (after cleaning):")
sample_counties_after = df_arcos.select("BUYER_COUNTY").unique().sort("BUYER_COUNTY").head(20)
print(sample_counties_after)

# Verify cleaning results
print("\n5. Verification:")
county_checks_after = df_arcos.select([
    (pl.col("BUYER_COUNTY").str.contains("(?i)county")).sum().alias("still_has_county_suffix"),
    pl.col("BUYER_COUNTY").n_unique().alias("unique_counties_after")
])
print(county_checks_after)

print(f"\n{'=' * 60}")
print("✓ County name cleaning complete!")

STEP 4: CLEAN COUNTY NAMES

1. Sample of ORIGINAL county names (before cleaning):
shape: (20, 1)
┌──────────────┐
│ BUYER_COUNTY │
│ ---          │
│ str          │
╞══════════════╡
│ null         │
│ ABBEVILLE    │
│ ACCOMACK     │
│ ADAMS        │
│ AIKEN        │
│ …            │
│ ALLENDALE    │
│ ALPINE       │
│ AMADOR       │
│ AMELIA       │
│ AMHERST      │
└──────────────┘

2. County Name Issues:
shape: (20, 1)
┌──────────────┐
│ BUYER_COUNTY │
│ ---          │
│ str          │
╞══════════════╡
│ null         │
│ ABBEVILLE    │
│ ACCOMACK     │
│ ADAMS        │
│ AIKEN        │
│ …            │
│ ALLENDALE    │
│ ALPINE       │
│ AMADOR       │
│ AMELIA       │
│ AMHERST      │
└──────────────┘

2. County Name Issues:
shape: (1, 3)
┌───────────────────┬────────────────┬────────────────────────┐
│ has_county_suffix ┆ has_whitespace ┆ unique_counties_before │
│ ---               ┆ ---            ┆ ---                    │
│ u32               ┆ u32            ┆ u32              

## Step 5: Handle Invalid/Missing Values

In [5]:
print("=" * 60)
print("STEP 5: HANDLE INVALID/MISSING VALUES")
print("=" * 60)

print(f"\n1. Initial row count: {df_arcos.shape[0]:,}")

# Check for null/missing values in critical columns
print("\n2. Checking for null values in critical columns:")
null_check = df_arcos.select([
    pl.col("BUYER_STATE").is_null().sum().alias("state_nulls"),
    pl.col("BUYER_COUNTY").is_null().sum().alias("county_nulls"),
    pl.col("year").is_null().sum().alias("year_nulls"),
    pl.col("MME").is_null().sum().alias("MME_nulls"),
    pl.col("DOSAGE_UNIT").is_null().sum().alias("DOSAGE_nulls")
])
print(null_check)

# Check for zero or negative values
print("\n3. Checking for invalid values (zero/negative):")
invalid_check = df_arcos.select([
    (pl.col("MME") <= 0).sum().alias("MME_invalid"),
    (pl.col("DOSAGE_UNIT") <= 0).sum().alias("DOSAGE_invalid"),
    ((pl.col("MME") <= 0) | (pl.col("DOSAGE_UNIT") <= 0)).sum().alias("either_invalid")
])
print(invalid_check)

# Filter out invalid records
print("\n4. Removing rows with invalid values...")
rows_before = df_arcos.shape[0]

df_arcos = df_arcos.filter(
    (pl.col("BUYER_STATE").is_not_null()) &
    (pl.col("BUYER_COUNTY").is_not_null()) &
    (pl.col("year").is_not_null()) &
    (pl.col("MME").is_not_null()) &
    (pl.col("DOSAGE_UNIT").is_not_null()) &
    (pl.col("MME") > 0) &
    (pl.col("DOSAGE_UNIT") > 0)
)

rows_after = df_arcos.shape[0]
rows_removed = rows_before - rows_after

print(f"   Rows before: {rows_before:,}")
print(f"   Rows after: {rows_after:,}")
print(f"   Rows removed: {rows_removed:,} ({(rows_removed/rows_before)*100:.2f}%)")

# Verify no invalid values remain
print("\n5. Verification - checking for remaining issues:")
verification = df_arcos.select([
    pl.col("BUYER_STATE").is_null().sum().alias("state_nulls"),
    pl.col("BUYER_COUNTY").is_null().sum().alias("county_nulls"),
    pl.col("year").is_null().sum().alias("year_nulls"),
    pl.col("MME").is_null().sum().alias("MME_nulls"),
    pl.col("DOSAGE_UNIT").is_null().sum().alias("DOSAGE_nulls"),
    (pl.col("MME") <= 0).sum().alias("MME_invalid"),
    (pl.col("DOSAGE_UNIT") <= 0).sum().alias("DOSAGE_invalid")
])
print(verification)

print(f"\n{'=' * 60}")
print("✓ Invalid values handled!")

STEP 5: HANDLE INVALID/MISSING VALUES

1. Initial row count: 218,477,461

2. Checking for null values in critical columns:
shape: (1, 5)
┌─────────────┬──────────────┬────────────┬───────────┬──────────────┐
│ state_nulls ┆ county_nulls ┆ year_nulls ┆ MME_nulls ┆ DOSAGE_nulls │
│ ---         ┆ ---          ┆ ---        ┆ ---       ┆ ---          │
│ u32         ┆ u32          ┆ u32        ┆ u32       ┆ u32          │
╞═════════════╪══════════════╪════════════╪═══════════╪══════════════╡
│ 0           ┆ 6914         ┆ 0          ┆ 45        ┆ 27015222     │
└─────────────┴──────────────┴────────────┴───────────┴──────────────┘

3. Checking for invalid values (zero/negative):
shape: (1, 3)
┌─────────────┬────────────────┬────────────────┐
│ MME_invalid ┆ DOSAGE_invalid ┆ either_invalid │
│ ---         ┆ ---            ┆ ---            │
│ u32         ┆ u32            ┆ u32            │
╞═════════════╪════════════════╪════════════════╡
│ 50          ┆ 2484495        ┆ 2484542        │
└──

## Step 6: Data Summary After Cleaning

In [6]:
print("=" * 60)
print("STEP 6: CLEANED DATA SUMMARY")
print("=" * 60)

# Overall statistics
print("\n1. Data Dimensions:")
print(f"   Total rows: {df_arcos.shape[0]:,}")
print(f"   Total columns: {df_arcos.shape[1]}")

# Year distribution
print("\n2. Year Distribution:")
year_dist = df_arcos.group_by("year").agg(
    pl.count().alias("record_count")
).sort("year")
print(year_dist)

# State distribution
print("\n3. State Distribution:")
state_dist = df_arcos.group_by("BUYER_STATE").agg(
    pl.count().alias("record_count")
).sort("BUYER_STATE")
print(state_dist)

# County counts by state
print("\n4. County Counts by State:")
county_by_state = df_arcos.group_by("BUYER_STATE").agg(
    pl.col("BUYER_COUNTY").n_unique().alias("unique_counties")
).sort("BUYER_STATE")
print(county_by_state)

# Summary statistics for key variables
print("\n5. MME and Dosage Summary Statistics:")
summary_stats = df_arcos.select([
    pl.col("MME").sum().alias("total_MME"),
    pl.col("MME").mean().alias("avg_MME"),
    pl.col("MME").median().alias("median_MME"),
    pl.col("DOSAGE_UNIT").sum().alias("total_dosage"),
    pl.col("DOSAGE_UNIT").mean().alias("avg_dosage"),
    pl.col("DOSAGE_UNIT").median().alias("median_dosage")
])
print(summary_stats)

# Sample of clean data
print("\n6. Sample of Cleaned Data (first 10 rows):")
print(df_arcos.select(["BUYER_STATE", "BUYER_COUNTY", "year", "MME", "DOSAGE_UNIT"]).head(10))

print(f"\n{'=' * 60}")
print("✓ Data is clean and ready for aggregation!")

STEP 6: CLEANED DATA SUMMARY

1. Data Dimensions:
   Total rows: 188,971,428
   Total columns: 11

2. Year Distribution:


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20512\1046779724.py:13: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("record_count")


shape: (10, 2)
┌──────┬──────────────┐
│ year ┆ record_count │
│ ---  ┆ ---          │
│ i32  ┆ u32          │
╞══════╪══════════════╡
│ 2006 ┆ 15310663     │
│ 2007 ┆ 16500532     │
│ 2008 ┆ 17560995     │
│ 2009 ┆ 18390776     │
│ 2010 ┆ 19240405     │
│ 2011 ┆ 20536778     │
│ 2012 ┆ 20853779     │
│ 2013 ┆ 21365622     │
│ 2014 ┆ 20163325     │
│ 2015 ┆ 19048553     │
└──────┴──────────────┘

3. State Distribution:


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20512\1046779724.py:20: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("record_count")


shape: (14, 2)
┌─────────────┬──────────────┐
│ BUYER_STATE ┆ record_count │
│ ---         ┆ ---          │
│ str         ┆ u32          │
╞═════════════╪══════════════╡
│ AL          ┆ 10603329     │
│ CA          ┆ 38640861     │
│ CO          ┆ 7812733      │
│ FL          ┆ 30004939     │
│ GA          ┆ 16320861     │
│ …           ┆ …            │
│ OR          ┆ 6803563      │
│ SC          ┆ 7526482      │
│ TN          ┆ 16179050     │
│ VA          ┆ 10603489     │
│ WA          ┆ 10917356     │
└─────────────┴──────────────┘

4. County Counts by State:
shape: (14, 2)
┌─────────────┬─────────────────┐
│ BUYER_STATE ┆ unique_counties │
│ ---         ┆ ---             │
│ str         ┆ u32             │
╞═════════════╪═════════════════╡
│ AL          ┆ 67              │
│ CA          ┆ 58              │
│ CO          ┆ 64              │
│ FL          ┆ 67              │
│ GA          ┆ 155             │
│ …           ┆ …               │
│ OR          ┆ 36              │
│ SC   

## Step 7: Aggregate to County-Year Level

In [7]:
# Aggregate to county-year level
print("Aggregating to county-year level...")
print("=" * 60)

# Group by state, county, and year; sum MME and DOSAGE_UNIT using Polars
df_county_year = (
    df_arcos
    .group_by(["BUYER_STATE", "BUYER_COUNTY", "year"])
    .agg([
        pl.col("MME").sum().alias("opioid_shipments_mme"),
        pl.col("DOSAGE_UNIT").sum().alias("total_pills")
    ])
)

# Rename columns for clarity
df_county_year = df_county_year.rename({
    "BUYER_STATE": "state",
    "BUYER_COUNTY": "county_name"
})

print(f"✓ Aggregation complete!")
print(f"  Aggregated rows: {df_county_year.shape[0]:,}")
print(f"  Columns: {df_county_year.shape[1]}")

print("\n" + "=" * 60)
print("Sample of aggregated data:")
print(df_county_year.head(20))

Aggregating to county-year level...
✓ Aggregation complete!
  Aggregated rows: 10,254
  Columns: 5

Sample of aggregated data:
shape: (20, 5)
┌───────┬─────────────┬──────┬──────────────────────┬─────────────┐
│ state ┆ county_name ┆ year ┆ opioid_shipments_mme ┆ total_pills │
│ ---   ┆ ---         ┆ ---  ┆ ---                  ┆ ---         │
│ str   ┆ str         ┆ i32  ┆ f64                  ┆ f64         │
╞═══════╪═════════════╪══════╪══════════════════════╪═════════════╡
│ GA    ┆ CHARLTON    ┆ 2010 ┆ 6.6356e6             ┆ 527090.0    │
│ GA    ┆ PULASKI     ┆ 2009 ┆ 1.8998e7             ┆ 1.117086e6  │
│ MS    ┆ CLARKE      ┆ 2012 ┆ 2.5171e7             ┆ 1.26492e6   │
│ GA    ┆ FULTON      ┆ 2010 ┆ 5.5771e9             ┆ 6.2861e8    │
│ MS    ┆ GEORGE      ┆ 2011 ┆ 2.1743e7             ┆ 1.9605e6    │
│ …     ┆ …           ┆ …    ┆ …                    ┆ …           │
│ CA    ┆ MODOC       ┆ 2007 ┆ 1.0941e7             ┆ 747379.8    │
│ OR    ┆ LINN        ┆ 2006 ┆ 1.0528e8   

## Step 8: Validate Aggregated Data Quality

In [8]:
# Validation checks
print("Data Quality Validation:")
print("=" * 60)

# Expected states (from original filtering)
expected_states = ['FL', 'WA', 'GA', 'AL', 'SC', 'NC', 'TN', 'MS', 'OR', 'CO', 'MN', 'NV', 'CA', 'VA']

# 1. Check for duplicates
duplicates = df_county_year.filter(pl.struct(["state", "county_name", "year"]).is_duplicated()).shape[0]
print(f"1. Duplicate rows: {duplicates}")
if duplicates > 0:
    print("   ⚠️  WARNING: Duplicates found!")
else:
    print("   ✓ No duplicates")

# 2. Verify year range
year_min = df_county_year["year"].min()
year_max = df_county_year["year"].max()
print(f"\n2. Year range: {year_min} - {year_max}")
if year_min == 2006 and year_max == 2015:
    print("   ✓ Correct year range (2006-2015)")
else:
    print(f"   ⚠️  Expected 2006-2015")

# 3. Confirm states present
unique_states = sorted(df_county_year["state"].unique().to_list())
print(f"\n3. States present ({len(unique_states)}):")
print(f"   {unique_states}")
missing_states = set(expected_states) - set(unique_states)
if missing_states:
    print(f"   ⚠️  Missing states: {missing_states}")
else:
    print(f"   ✓ All {len(expected_states)} expected states present")

# 4. Check for missing values
print(f"\n4. Missing values:")
null_counts = df_county_year.null_count()
total_nulls = sum(null_counts.row(0))
if total_nulls == 0:
    print("   ✓ No missing values")
else:
    print(null_counts)

# 5. Summary statistics
print(f"\n5. Summary statistics:")
print(f"   Total counties: {df_county_year['county_name'].n_unique()}")
print(f"   Total state-county-year observations: {df_county_year.shape[0]:,}")
print(f"   Years per county (avg): {df_county_year.shape[0] / df_county_year['county_name'].n_unique():.1f}")

print("\n" + "=" * 60)
print("Validation complete!")

Data Quality Validation:
1. Duplicate rows: 0
   ✓ No duplicates

2. Year range: 2006 - 2015
   ✓ Correct year range (2006-2015)

3. States present (14):
   ['AL', 'CA', 'CO', 'FL', 'GA', 'MN', 'MS', 'NC', 'NV', 'OR', 'SC', 'TN', 'VA', 'WA']
   ✓ All 14 expected states present

4. Missing values:
   ✓ No missing values

5. Summary statistics:
   Total counties: 774
   Total state-county-year observations: 10,254
   Years per county (avg): 13.2

Validation complete!


## Step 9: Save Final Processed Data to Parquet

In [9]:
# Create output directory if it doesn't exist
output_dir = 'data/intermediate'
os.makedirs(output_dir, exist_ok=True)

# Save to parquet
output_file = os.path.join(output_dir, 'arcos_county_year.parquet')

print(f"Saving final processed data to: {output_file}")
print("=" * 60)

df_county_year.write_parquet(output_file, compression='snappy')

# Check file size
file_size = os.path.getsize(output_file)
print(f"✓ File saved successfully!")
print(f"  File: {output_file}")
print(f"  Size: {file_size:,} bytes ({file_size / 1024:.2f} KB)")
print(f"  Rows: {df_county_year.shape[0]:,}")
print(f"  Columns: {df_county_year.shape[1]}")

print("\n" + "=" * 60)
print("Final Dataset Summary:")
print("=" * 60)
print(f"  Time period: {df_county_year['year'].min()} - {df_county_year['year'].max()}")
print(f"  States: {df_county_year['state'].n_unique()}")
print(f"  Counties: {df_county_year['county_name'].n_unique()}")
print(f"  Total observations: {df_county_year.shape[0]:,}")
print(f"\n  Columns:")
for col in df_county_year.columns:
    print(f"    - {col}")

print("\n✓ Preprocessing complete! Data ready for analysis.")

Saving final processed data to: data/intermediate\arcos_county_year.parquet
✓ File saved successfully!
  File: data/intermediate\arcos_county_year.parquet
  Size: 186,308 bytes (181.94 KB)
  Rows: 10,254
  Columns: 5

Final Dataset Summary:
  Time period: 2006 - 2015
  States: 14
  Counties: 774
  Total observations: 10,254

  Columns:
    - state
    - county_name
    - year
    - opioid_shipments_mme
    - total_pills

✓ Preprocessing complete! Data ready for analysis.


## Step 10: Additional Data Quality Checks (Post-Analysis)

In [10]:
print("=" * 60)
print("ADDITIONAL DATA QUALITY CHECKS")
print("=" * 60)

# 1. Check for counties with incomplete time series
print("\n1. Counties with Incomplete Time Series:")
print("   (Counties that don't have all 10 years of data)")

county_year_counts = df_county_year.group_by(["state", "county_name"]).agg(
    pl.count().alias("years_present")
).filter(pl.col("years_present") < 10).sort(["state", "county_name"])

print(f"   Counties with < 10 years: {county_year_counts.shape[0]}")
if county_year_counts.shape[0] > 0:
    print(f"   First 20 incomplete counties:")
    print(county_year_counts.head(20))
else:
    print("   ✓ All counties have complete 10-year data!")

# 2. Check for extreme outliers in MME and pills
print("\n2. Extreme Outliers Check:")
outlier_stats = df_county_year.select([
    pl.col("opioid_shipments_mme").quantile(0.99).alias("MME_99th_percentile"),
    pl.col("opioid_shipments_mme").max().alias("MME_max"),
    pl.col("total_pills").quantile(0.99).alias("pills_99th_percentile"),
    pl.col("total_pills").max().alias("pills_max")
])
print(outlier_stats)

# Show top 10 counties by MME
print("\n   Top 10 counties by total opioid shipments (MME):")
top_mme = df_county_year.sort("opioid_shipments_mme", descending=True).head(10)
print(top_mme)

# 3. Check for counties with zero shipments (shouldn't exist after filtering)
print("\n3. Check for Zero Shipments:")
zero_shipments = df_county_year.filter(
    (pl.col("opioid_shipments_mme") == 0) | (pl.col("total_pills") == 0)
)
print(f"   Counties with zero shipments: {zero_shipments.shape[0]}")
if zero_shipments.shape[0] > 0:
    print("   ⚠️  WARNING: Found counties with zero shipments!")
    print(zero_shipments.head(10))
else:
    print("   ✓ No zero shipments found!")

# 4. Check for potential data entry errors (MME to Pills ratio)
print("\n4. MME to Pills Ratio Check:")
print("   (Checking for unusual ratios that might indicate data issues)")

df_with_ratio = df_county_year.with_columns([
    (pl.col("opioid_shipments_mme") / pl.col("total_pills")).alias("mme_per_pill")
])

ratio_stats = df_with_ratio.select([
    pl.col("mme_per_pill").min().alias("min_ratio"),
    pl.col("mme_per_pill").quantile(0.01).alias("1st_percentile"),
    pl.col("mme_per_pill").median().alias("median_ratio"),
    pl.col("mme_per_pill").quantile(0.99).alias("99th_percentile"),
    pl.col("mme_per_pill").max().alias("max_ratio")
])
print(ratio_stats)

# Show extreme ratios
extreme_low = df_with_ratio.filter(pl.col("mme_per_pill") < 0.1).sort("mme_per_pill").head(5)
extreme_high = df_with_ratio.filter(pl.col("mme_per_pill") > 100).sort("mme_per_pill", descending=True).head(5)

if extreme_low.shape[0] > 0:
    print(f"\n   Unusually LOW MME/pill ratios (< 0.1): {extreme_low.shape[0]} counties")
    print(extreme_low.select(["state", "county_name", "year", "opioid_shipments_mme", "total_pills", "mme_per_pill"]))
else:
    print("\n   ✓ No unusually low MME/pill ratios found")

if extreme_high.shape[0] > 0:
    print(f"\n   Unusually HIGH MME/pill ratios (> 100): {extreme_high.shape[0]} counties")
    print(extreme_high.select(["state", "county_name", "year", "opioid_shipments_mme", "total_pills", "mme_per_pill"]))
else:
    print("\n   ✓ No unusually high MME/pill ratios found")

# 5. Summary of data completeness by state
print("\n5. Data Completeness by State:")
state_summary = df_county_year.group_by("state").agg([
    pl.col("county_name").n_unique().alias("unique_counties"),
    pl.count().alias("total_observations"),
    (pl.count() / pl.col("county_name").n_unique()).alias("avg_years_per_county")
]).sort("state")
print(state_summary)

print("\n" + "=" * 60)
print("RECOMMENDATION SUMMARY")
print("=" * 60)
print("\nBased on the checks above:")
print("1. Review counties with incomplete time series (if any)")
print("2. Investigate extreme outliers (top counties by MME)")
print("3. Check unusual MME/pill ratios for potential data quality issues")
print("4. Consider if any further filtering is needed before analysis")
print("\n" + "=" * 60)

ADDITIONAL DATA QUALITY CHECKS

1. Counties with Incomplete Time Series:
   (Counties that don't have all 10 years of data)
   Counties with < 10 years: 21
   First 20 incomplete counties:
shape: (20, 3)
┌───────┬────────────────┬───────────────┐
│ state ┆ county_name    ┆ years_present │
│ ---   ┆ ---            ┆ ---           │
│ str   ┆ str            ┆ u32           │
╞═══════╪════════════════╪═══════════════╡
│ CA    ┆ ALPINE         ┆ 6             │
│ CO    ┆ COSTILLA       ┆ 1             │
│ CO    ┆ CUSTER         ┆ 8             │
│ CO    ┆ MINERAL        ┆ 1             │
│ CO    ┆ SAGUACHE       ┆ 9             │
│ …     ┆ …              ┆ …             │
│ TN    ┆ VAN BUREN      ┆ 6             │
│ VA    ┆ EMPORIA CITY   ┆ 1             │
│ VA    ┆ KING AND QUEEN ┆ 1             │
│ VA    ┆ ROCKBRIDGE     ┆ 1             │
│ VA    ┆ SURRY          ┆ 9             │
└───────┴────────────────┴───────────────┘

2. Extreme Outliers Check:
shape: (1, 4)
┌─────────────────────┬

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20512\1229106989.py:10: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("years_present")
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20512\1229106989.py:84: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("total_observations"),
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20512\1229106989.py:85: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  (pl.count() / pl.col("county_name").n_unique()).alias("avg_years_per_county")


## Step 11: Apply Additional Cleaning Based on Quality Checks

In [11]:
print("=" * 60)
print("STEP 11: APPLY ADDITIONAL CLEANING")
print("=" * 60)

print(f"\nInitial rows: {df_county_year.shape[0]:,}")
print(f"Initial counties: {df_county_year.select([pl.col('state'), pl.col('county_name')]).unique().shape[0]:,}")

# 1. Remove counties with incomplete time series (< 10 years)
print("\n1. Removing counties with incomplete time series...")

# Get counties with complete data (10 years)
counties_with_complete_data = (
    df_county_year
    .group_by(["state", "county_name"])
    .agg(pl.len().alias("year_count"))
    .filter(pl.col("year_count") == 10)
    .select(["state", "county_name"])
)

df_county_year_clean = df_county_year.join(
    counties_with_complete_data,
    on=["state", "county_name"],
    how="inner"
)

removed_incomplete = df_county_year.shape[0] - df_county_year_clean.shape[0]
print(f"   Removed {removed_incomplete:,} observations from counties with incomplete data")
print(f"   Remaining rows: {df_county_year_clean.shape[0]:,}")

# 2. Flag/investigate extreme outliers (optional - for review only)
print("\n2. Flagging extreme outliers (for manual review)...")

# Calculate MME per pill ratio
df_county_year_clean = df_county_year_clean.with_columns([
    (pl.col("opioid_shipments_mme") / pl.col("total_pills")).alias("mme_per_pill")
])

# Get percentiles for outlier detection
p99_mme = df_county_year_clean.select(pl.col("opioid_shipments_mme").quantile(0.99)).item()
p99_ratio = df_county_year_clean.select(pl.col("mme_per_pill").quantile(0.99)).item()

print(f"   99th percentile MME: {p99_mme:,.0f}")
print(f"   99th percentile MME/pill ratio: {p99_ratio:.2f}")

# Create flags but don't remove (let analyst decide)
df_county_year_clean = df_county_year_clean.with_columns([
    (pl.col("opioid_shipments_mme") > p99_mme).alias("extreme_mme_flag"),
    (pl.col("mme_per_pill") > 100).alias("unusual_ratio_flag")
])

extreme_mme_count = df_county_year_clean.filter(pl.col("extreme_mme_flag")).shape[0]
unusual_ratio_count = df_county_year_clean.filter(pl.col("unusual_ratio_flag")).shape[0]

print(f"   Flagged {extreme_mme_count:,} observations with extreme MME (>99th percentile)")
print(f"   Flagged {unusual_ratio_count:,} observations with unusual MME/pill ratio (>100)")
print(f"   NOTE: These are FLAGGED for review, not removed")

# 3. Option to remove extreme outliers (OPTIONAL - commented out by default)
print("\n3. Option to Remove Extreme Outliers:")
print("   DECISION: Keep all data with flags for analyst to decide")
print("   To remove outliers, uncomment the code below and rerun")

# Uncomment these lines to remove extreme outliers:
# df_county_year_clean = df_county_year_clean.filter(
#     ~pl.col("extreme_mme_flag") & ~pl.col("unusual_ratio_flag")
# )
# print(f"   After removing outliers: {df_county_year_clean.shape[0]:,} observations")

# 4. Final summary
print("\n4. Final Cleaned Dataset Summary:")
print(f"   Total observations: {df_county_year_clean.shape[0]:,}")
print(f"   Unique counties: {df_county_year_clean.select([pl.col('state'), pl.col('county_name')]).unique().shape[0]:,}")
print(f"   States: {df_county_year_clean['state'].n_unique()}")
print(f"   Years: {df_county_year_clean['year'].min()} - {df_county_year_clean['year'].max()}")

# Show state breakdown
state_breakdown = df_county_year_clean.group_by("state").agg([
    pl.col("county_name").n_unique().alias("counties"),
    pl.len().alias("observations")
]).sort("state")
print("\n   Counties per state:")
print(state_breakdown)

print("\n" + "=" * 60)
print("✓ Additional cleaning complete!")
print("=" * 60)

STEP 11: APPLY ADDITIONAL CLEANING

Initial rows: 10,254
Initial counties: 1,038

1. Removing counties with incomplete time series...
   Removed 84 observations from counties with incomplete data
   Remaining rows: 10,170

2. Flagging extreme outliers (for manual review)...
   99th percentile MME: 11,852,123,218
   99th percentile MME/pill ratio: 43.36
   Flagged 102 observations with extreme MME (>99th percentile)
   Flagged 31 observations with unusual MME/pill ratio (>100)
   NOTE: These are FLAGGED for review, not removed

3. Option to Remove Extreme Outliers:
   DECISION: Keep all data with flags for analyst to decide
   To remove outliers, uncomment the code below and rerun

4. Final Cleaned Dataset Summary:
   Total observations: 10,170
   Unique counties: 1,017
   States: 14
   Years: 2006 - 2015

   Counties per state:
shape: (14, 3)
┌───────┬──────────┬──────────────┐
│ state ┆ counties ┆ observations │
│ ---   ┆ ---      ┆ ---          │
│ str   ┆ u32      ┆ u32          │
╞